# Fourier BSDF by Wenzel Jakob

W. Jakob, E. d'Eon, O. Jakob, S. Marschner, *A comprehensive framework for rendering layered materials*. ACM Trans. Graph. 33, 4, Article 118 (July 2014), 14 pages. https://doi.org/10.1145/2601097.2601139

In [ ]:
from liar.shaders import Jakob

# bsdf = Jakob(r"D:\coding\pbrt-v3-scenes\dragon\bsdfs\coated_copper.bsdf")
bsdf = Jakob(r"D:\coding\pbrt-v3-scenes\barcelona-pavilion\bsdfs\leather.bsdf")

# list the nodes
print(bsdf.nodes)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_a0(bsdf):
    nodes = bsdf.nodes
    max_neg = max(i for i in range(len(nodes)) if nodes[i] < 0)
    min_pos = min(i for i in range(len(nodes)) if nodes[i] > 0)
    assert min_pos == max_neg + 3, (
        f"{max_neg=}: {nodes[max_neg]=}, {min_pos=}: {nodes[min_pos]=}"
    )  # because of how the Jakob BSDF is defined
    neg_indices = list(range(0, max_neg + 2))
    pos_indices = list(range(min_pos - 1, len(nodes)))
    neg_mu = np.array(nodes[: max_neg + 2])
    pos_mu = np.array(nodes[min_pos - 1 :])
    assert neg_mu[-1] == 0, neg_mu[-1]
    assert pos_mu[0] == 0, pos_mu[0]

    def plot_quadrant(mu_i, mu_o, index_i, index_o):
        a0 = np.zeros((len(index_i), len(index_o)))
        for i, ii in enumerate(index_i):
            for o, oo in enumerate(index_o):
                coeffs = bsdf.coefficients(ii, oo, 0)
                a0[o, i] = (coeffs[0] if coeffs else 0) * abs(mu_i[i] * mu_o[o])
        plt.pcolormesh(
            mu_i, mu_o, a0, vmin=0, vmax=np.percentile(a0, 99), shading="nearest"
        )
        plt.gca().invert_yaxis()
        plt.colorbar()

    fig = plt.figure(tight_layout=True)
    fig.add_subplot(2, 2, 1)
    plt.title("$Tˆ{bt}$")
    plot_quadrant(neg_mu, neg_mu, neg_indices, neg_indices)
    fig.add_subplot(2, 2, 2)
    plt.title("$Rˆ{t}$")
    plot_quadrant(pos_mu, neg_mu, pos_indices, neg_indices)
    fig.add_subplot(2, 2, 3)
    plt.title("$Rˆ{b}$")
    plot_quadrant(neg_mu, pos_mu, neg_indices, pos_indices)
    fig.add_subplot(2, 2, 4)
    plt.title("$Tˆ{tb}$")
    plot_quadrant(pos_mu, pos_mu, pos_indices, pos_indices)
    plt.show()


plot_a0(bsdf)